In [1]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from data_clean import trimnoise

In [18]:
df = pd.read_csv('testWalk1.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.rename(columns={'Speed (m/s)': "speed"})
df = df.drop(df[df.Latitude == 0.0].index)
df_filtered = trimnoise(df)
df_filtered['time'] = df_filtered['time'].diff()
df_filtered['speed'] = df_filtered['speed'].diff()

In [19]:
groupSize = 32
grouped = df_filtered.groupby(df_filtered.index // groupSize)

In [23]:
df_grouped = grouped.mean()
#df_mean.drop(columns=['gFx', 'gFy', 'gFz'])
df_grouped = df_grouped[['time','speed','ax_f', 'ay_f', 'az_f','wx_f', 'wy_f', 'wz_f','gFx_f', 'gFy_f', 'gFz_f',]]

In [24]:
df_grouped['time'] = grouped.sum()['time']

In [25]:
df_grouped

,time,speed,ax_f,ay_f,az_f,wx_f,wy_f,wz_f,gFx_f,gFy_f,gFz_f
0,0.348,0.000000,0.021751,-0.001245,0.004898,-0.030297,0.095843,0.051645,0.031599,0.434258,0.917375
1,0.356,0.000000,0.006868,0.002951,-0.028643,-0.046384,0.175446,0.084065,-0.001827,0.422979,0.917697
2,0.412,0.000000,-0.008826,0.008636,0.044071,-0.121598,0.176885,0.125961,-0.052831,0.389259,0.937475
3,0.377,0.000000,-0.163437,0.064158,-0.015004,-0.007188,0.018342,0.105971,-0.087640,0.387736,0.939503
4,0.390,0.000000,-0.551367,0.354245,-0.596050,0.020960,0.174556,0.283107,-0.131024,0.431525,0.854480
...,...,...,...,...,...,...,...,...,...,...,...
2161,0.377,0.000000,0.387490,-0.010106,1.784541,-0.137495,-0.294734,-0.042702,0.220056,0.323009,1.013699
2162,0.400,0.000625,-0.509422,0.738972,-0.135907,-0.161012,-0.131820,-0.545522,0.139422,0.415785,0.869236
2163,0.399,0.000000,-1.055886,-0.047007,-0.501131,0.041138,0.147235,0.210916,0.027821,0.311405,0.868698
2164,0.383,0.000000,0.856339,-0.235929,1.276139,0.092378,-0.163870,-0.057332,0.253862,0.369023,1.008336


In [109]:
df_grouped.loc[df_grouped['speed'] > 0.0]

,time,speed,ax_f,ay_f,az_f,wx_f,wy_f,wz_f,gFx_f,gFy_f,gFz_f
10,0.412,0.038125,0.214933,-0.835303,0.450165,0.024097,-0.045051,0.112511,-0.058170,0.539145,0.885546
13,0.355,0.008125,0.035306,-0.556406,0.454154,0.131091,-0.145837,-0.148950,-0.044675,0.408759,0.934362
15,0.352,0.004375,0.055244,0.021420,0.084085,0.395806,-0.144910,0.145122,-0.009748,0.569251,0.811008
23,0.352,0.001563,0.601144,0.320297,-0.962388,0.147493,0.021614,0.349276,1.037055,-0.112992,0.109767
26,0.379,0.000937,0.843945,-0.782470,0.421919,-0.251150,0.211569,-0.174363,1.105255,-0.196431,0.243368
...,...,...,...,...,...,...,...,...,...,...,...
2118,0.393,0.000312,-0.154854,-0.929672,0.946816,0.153143,0.187077,-0.117217,0.931367,-0.291280,-0.187217
2134,0.383,0.000312,0.790321,-1.000437,0.341652,-0.088185,-0.061051,0.047123,1.037252,-0.365815,-0.156594
2147,0.400,0.000312,0.553337,0.686382,-0.831605,-0.232458,0.041606,-0.105169,0.927096,-0.162574,-0.303720
2162,0.400,0.000625,-0.509422,0.738972,-0.135907,-0.161012,-0.131820,-0.545522,0.139422,0.415785,0.869236


In [98]:
from sklearn.model_selection import train_test_split

X = df_grouped[['time', 'ax_f', 'ay_f','az_f', 'wx_f', 'wy_f','wz_f']].to_numpy()
y = df_grouped['speed'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [99]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
degree=7
polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())

In [100]:
result = polyreg.fit(X,y)

In [101]:
result.score(X_test, y_test)

0.999999999999999

In [113]:
x_check = df_grouped.iloc[26][['time', 'ax_f', 'ay_f','az_f', 'wx_f', 'wy_f','wz_f']].to_numpy()

In [114]:
df_grouped.iloc[26]['speed']

0.0009374990625000031

In [115]:
result.predict([x_check])

array([0.0009375])